In [1]:
cd '/Users/joshherzberg/Documents/chihack/article-tagging/lib/'

/Users/joshherzberg/Documents/chihack/article-tagging/lib


In [2]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import os
import tagnews
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import json
import requests
import keras
import shutil

In [4]:
glove = tagnews.load_glove('tagnews/data/glove.6B.50d.txt')

In [5]:
glove.shape

(400000, 50)

In [6]:
glove.loc['address_vec'] = glove.loc[['street', 'avenue', 'place', 'road', 'block', 'main', 'city', 'west', 'east', 'north', 'south']].mean()
glove.loc['neighborhood_vec'] = glove.loc[['neighborhood', 'burrough', 'community', 'area']].mean()

In [7]:
with open('tagnews/data/Chicago_Street_Names.csv') as street_names:
    streets = street_names.read().splitlines()[1:]
streets = [i.lower() for i in streets]

with open('tagnews/data/chicago_neighborhoods.csv') as neighborhoods:
    hoods = neighborhoods.read().splitlines()
hoods = list(set([j.lower().replace('\"', '') for j in hoods]))

for name in streets:
    glove.loc[name] = glove.loc['address_vec']
for hood in hoods:
    glove.loc[hood] = glove.loc['neighborhood_vec']

In [8]:
with open('tagnews/data/training.txt', encoding='utf-8') as f:
    our_training_data = f.read()
    
training_df = pd.DataFrame([x.split() for x in our_training_data.split('\n') if x],
                           columns=['word', 'tag'])

training_df.iloc[:,1] = training_df.iloc[:,1].apply(int)
training_df['all_tags'] = 'NA'

# If you want to join our data w/ kaggle data, you can do this.
# ner = tagnews.load_ner_data('tagnews/data/')
# pd.concat([training_df, ner]).reset_index(drop=True)

# If you just want to use our data, you can do this.
ner = training_df

ner = ner[['word', 'all_tags', 'tag']]

In [9]:
# pd.DataFrame(glove.loc[ner.loc[ner['word'] == 'Woodlawn']['word'].str.lower()].values)

In [10]:
ner = pd.concat([ner,
                 pd.DataFrame(ner['word'].str[0].str.isupper().values),
                 pd.DataFrame(glove.loc[ner['word'].str.lower()].values),
                 pd.DataFrame(ner['word'].str.isnumeric().values),
                 pd.DataFrame(ner['word'].str.len().values)],
                 axis='columns')
ner.fillna(value=0.0, inplace=True)

data_dim = 53
timesteps = 25 # only during training, testing can take arbitrary length.
num_classes = 2

train_val_split = int(19 * ner.shape[0] / 20.)

ner_train_idxs = range(0, train_val_split - timesteps, timesteps)
x_train = np.array([ner.iloc[i:i+timesteps, 3:].values
                    for i in ner_train_idxs])
y_train = np.array([to_categorical(ner.iloc[i:i+timesteps, 2].values, 2)
                    for i in ner_train_idxs])

ner_val_idxs = range(train_val_split, ner.shape[0] - timesteps, timesteps)
x_val = np.array([ner.iloc[i:i+timesteps, 3:].values
                  for i in ner_val_idxs])
y_val = np.array([to_categorical(ner.iloc[i:i+timesteps, 2].values, 2)
                  for i in ner_val_idxs])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(None, data_dim)))
model.add(LSTM(8, return_sequences=True))
model.add(TimeDistributed(Dense(2, activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])
print(model.summary(100))

____________________________________________________________________________________________________
Layer (type)                                 Output Shape                            Param #        
lstm_1 (LSTM)                                (None, None, 32)                        11008          
____________________________________________________________________________________________________
lstm_2 (LSTM)                                (None, None, 8)                         1312           
____________________________________________________________________________________________________
time_distributed_1 (TimeDistributed)         (None, None, 2)                         18             
Total params: 12,338
Trainable params: 12,338
Non-trainable params: 0
____________________________________________________________________________________________________
None


In [15]:
os.makedirs('tmp', exist_ok=True)
checkpointer = ModelCheckpoint(filepath='./tmp/weights-{epoch:02d}.hdf5',
                               monitor='val_categorical_accuracy',
                               verbose=1,
                               save_best_only=True)

class OurAUC(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        # Go to https://geo-extract-tester.herokuapp.com/ and download
        # the validation data (validation.txt).
        '''with open('validation.txt', encoding='utf-8') as f:
            s = f.read()

        gloved_data = pd.concat([pd.DataFrame([[w[0].isupper()] for w in s.split('\n') if w]),
                                 glove.loc[[w for w in s.split('\n') if w]].fillna(0).reset_index(drop=True)],
                                 axis='columns')
        glove_time_size = 100
        preds_batched = []
        i = 0
        while gloved_data[i:i+glove_time_size].size:
            preds_batched.append(model.predict(np.expand_dims(gloved_data[i:i+glove_time_size],
                                                              axis=0))[0][:,1])
            i += glove_time_size

        with open('guesses-{epoch:02d}.txt'.format(epoch=epoch), 'w') as f:
            for prob in [p for pred in preds_batched for p in pred]:
                f.write(str(prob) + '\n')'''

        with open('tagnews/data/validation.txt', encoding='utf-8') as f:
            s = f.read()

        gloved_data = pd.concat([pd.DataFrame([[w[0].isupper()] for w in s.split('\n') if w]),
                                 glove.loc[[w for w in s.split('\n') if w]].fillna(0).reset_index(drop=True),
                                 pd.DataFrame([[w[0].isnumeric()] for w in s.split('\n') if w]),
                                 pd.DataFrame([[len(w[0])] for w in s.split('\n') if w])],
                                 axis='columns')
        glove_time_size = 100
        preds_batched = []
        i = 0
        while gloved_data[i:i+glove_time_size].size:
            preds_batched.append(model.predict(np.expand_dims(gloved_data[i:i+glove_time_size],
                                                              axis=0))[0][:,1])
            i += glove_time_size

        with open('guesses-{epoch:02d}.txt'.format(epoch=epoch), 'w') as f:
            for prob in [p for pred in preds_batched for p in pred]:
                f.write(str(prob) + '\n')

        with open('guesses-{epoch:02d}.txt'.format(epoch=epoch), 'rb') as f:
            url = 'https://geo-extract-tester.herokuapp.com/api/score'
            r = requests.post(url, files={'file': f})
            try:
                print('AUC: {:.5f}'.format(json.loads(r.text)['auc']))
            except KeyError:
                raise ValueError('Problem retrieving AUC from API. Is your validation set up to date?')

our_auc = OurAUC()

In [16]:
model.fit(x_train, y_train,
          epochs=20,
          validation_data=(x_val, y_val),
          callbacks=[checkpointer, our_auc],
          verbose=2)

Train on 6961 samples, validate on 366 samples
Epoch 1/20
 - 7s - loss: 0.0786 - categorical_accuracy: 0.9685 - val_loss: 0.0670 - val_categorical_accuracy: 0.9793

Epoch 00001: val_categorical_accuracy improved from -inf to 0.97934, saving model to ./tmp/weights-01.hdf5


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


AUC: 0.95681
Epoch 2/20
 - 7s - loss: 0.0670 - categorical_accuracy: 0.9752 - val_loss: 0.0624 - val_categorical_accuracy: 0.9813

Epoch 00002: val_categorical_accuracy improved from 0.97934 to 0.98131, saving model to ./tmp/weights-02.hdf5
AUC: 0.96140
Epoch 3/20
 - 7s - loss: 0.0629 - categorical_accuracy: 0.9762 - val_loss: 0.0654 - val_categorical_accuracy: 0.9812

Epoch 00003: val_categorical_accuracy did not improve from 0.98131
AUC: 0.96158
Epoch 4/20
 - 7s - loss: 0.0609 - categorical_accuracy: 0.9769 - val_loss: 0.0605 - val_categorical_accuracy: 0.9820

Epoch 00004: val_categorical_accuracy improved from 0.98131 to 0.98197, saving model to ./tmp/weights-04.hdf5
AUC: 0.96314
Epoch 5/20
 - 7s - loss: 0.0593 - categorical_accuracy: 0.9773 - val_loss: 0.0581 - val_categorical_accuracy: 0.9816

Epoch 00005: val_categorical_accuracy did not improve from 0.98197
AUC: 0.96532
Epoch 6/20
 - 7s - loss: 0.0579 - categorical_accuracy: 0.9776 - val_loss: 0.0595 - val_categorical_accuracy: